# FCN

Fully Connected Network

## GPU Check

In [1]:
# # check available GPU in terminal: watch -d -n 0.5 nvidia-smi
# # select one GPU to train on:
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="2"

## Imports

In [2]:
import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../modules'))
import notebook_loading

from bn_FCN_Train import bn_FCN_Train
from FCN_Data import *
from bn_FC_Network import bn_FC_Network

importing Jupyter notebook from bn_FCN_Train.ipynb
importing Jupyter notebook from FCN_Data.ipynb
importing Jupyter notebook from bn_FC_Network.ipynb


## Load and Setup Data

In [3]:
# extract the redundant baselines and their gains and data from miriad and calfits files
red_bls, gains, uvd = load_relevant_data('../zen_data/zen.2458098.58037.xx.HH.uv','../zen_data/zen.2458098.58037.xx.HH.uv.abs.calfits')

# seperate trining and testing redundant baselines 
# if we have not already done this, load them from disk
training_redundant_baselines_dict, testing_redundant_baselines_dict = get_or_gen_test_train_red_bls_dicts(red_bls, gains.keys())

# seperate the visiblites
training_baselines_data = get_seps_data(training_redundant_baselines_dict, uvd)
testing_baselines_data = get_seps_data(testing_redundant_baselines_dict, uvd)

## Trainer

In [4]:
trainer = bn_FCN_Train(num_flatnesses=100)

In [5]:
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
                 (testing_baselines_data, testing_redundant_baselines_dict),
                 gains)

## Network

In [10]:
layer_nodes = [512,256,128,128,64,64,32]
test_1 = bn_FC_Network(name = 'test_1', layer_nodes = layer_nodes)
test_1.initial_learning_rate = 0.001
test_1.cost_name = 'MSE'
test_1.g_shift = 1e-5
test_1.print_params()


{'cost_name': 'MSE',
 'g_shift': 1e-05,
 'initial_learning_rate': 0.001,
 'layer_nodes': [512, 256, 128, 128, 64, 64, 32],
 'log_dir': 'logs/',
 'name': 'test_1',
 'num_freq_channels': 1024,
 'threshold': 0.00625}


## Train

In [11]:
trainer.network = test_1
trainer.fcl_keep_prob = 0.50
trainer.sample_keep_prob = 1.0
trainer.batch_size = 16
trainer.num_epochs = 100 # quick example
trainer.model_save_interval = 25
trainer.train()

Training Finished


In [9]:
layer_nodes = [512,256,128,128,64,64,32]
test_2 = bn_FC_Network(name = 'test_2', layer_nodes = layer_nodes)
test_2.load_params('logs/test_1/params/bn_FC_Network')
test_2.initial_learning_rate = 0.00001
test_2.cost_name = 'MISG'
test_2.g_shift = 1e-5
test_2.print_params()

trainer.network = test_2
trainer.pretrained_model_path = 'logs/test_1/trained_model.ckpt-99'
trainer.fcl_keep_prob = 0.50
trainer.sample_keep_prob = 1.0
trainer.batch_size = 32
trainer.num_epochs = 100 # quick example
trainer.model_save_interval = 25
trainer.train()

Loading Netowrk Parameters{'cost_name': 'MISG',
 'g_shift': 1e-05,
 'initial_learning_rate': 1e-05,
 'layer_nodes': [512, 256, 128, 128, 64, 64, 32],
 'log_dir': 'logs/',
 'name': 'test_2',
 'num_freq_channels': 1024,
 'threshold': 0.00625}
Creating Network GraphINFO:tensorflow:Restoring parameters from logs/test_1/trained_model.ckpt-99
Training Finished


In [14]:
layer_nodes = [512,256,128,128,64,64,32]
test_3 = bn_FC_Network(name = 'test_3', layer_nodes = layer_nodes)
test_3.load_params('logs/test_2/params/bn_FC_Network')
test_3.initial_learning_rate = 0.00001
test_3.cost_name = 'MISG'
test_3.g_shift = 1e-5
test_3.print_params()

trainer.network = test_3
trainer.pretrained_model_path = 'logs/test_2/trained_model.ckpt-99'
trainer.fcl_keep_prob = 0.50
trainer.sample_keep_prob = 1.0
trainer.batch_size = 64
trainer.num_epochs = 100 # quick example
trainer.model_save_interval = 25
trainer.train()

Loading Netowrk Parameters{'cost_name': 'MISG',
 'g_shift': 1e-05,
 'initial_learning_rate': 1e-05,
 'layer_nodes': [512, 256, 128, 128, 64, 64, 32],
 'log_dir': 'logs/',
 'name': 'test_3',
 'num_freq_channels': 1024,
 'threshold': 0.00625}
Creating Network GraphINFO:tensorflow:Restoring parameters from logs/test_2/trained_model.ckpt-99
Training Finished


In [15]:
layer_nodes = [512,256,128,128,64,64,32]
test_4 = bn_FC_Network(name = 'test_4', layer_nodes = layer_nodes)
test_4.load_params('logs/test_3/params/bn_FC_Network')
test_4.initial_learning_rate = 0.0001
test_4.cost_name = 'MISG'
test_4.g_shift = 1e-5
test_4.print_params()

trainer.network = test_4
trainer.pretrained_model_path = 'logs/test_3/trained_model.ckpt-99'
trainer.fcl_keep_prob = 0.50
trainer.sample_keep_prob = 1.0
trainer.batch_size = 64
trainer.num_epochs = 100 # quick example
trainer.model_save_interval = 25
trainer.train()

Loading Netowrk Parameters{'cost_name': 'MISG',
 'g_shift': 1e-05,
 'initial_learning_rate': 0.0001,
 'layer_nodes': [512, 256, 128, 128, 64, 64, 32],
 'log_dir': 'logs/',
 'name': 'test_4',
 'num_freq_channels': 1024,
 'threshold': 0.00625}
Creating Network GraphINFO:tensorflow:Restoring parameters from logs/test_3/trained_model.ckpt-99
Training Finished


In [16]:
layer_nodes = [512,256,128,128,64,64,32]
test_5 = bn_FC_Network(name = 'test_5', layer_nodes = layer_nodes)
test_5.load_params('logs/test_4/params/bn_FC_Network')
test_5.initial_learning_rate = 0.0001
test_5.cost_name = 'MISG'
test_5.g_shift = 1e-5
test_5.print_params()

trainer.network = test_5
trainer.pretrained_model_path = 'logs/test_4/trained_model.ckpt-99'
trainer.fcl_keep_prob = 0.50
trainer.sample_keep_prob = 1.0
trainer.batch_size = 64
trainer.num_epochs = 10000 # quick example
trainer.model_save_interval = 25
trainer.train()

Loading Netowrk Parameters{'cost_name': 'MISG',
 'g_shift': 1e-05,
 'initial_learning_rate': 0.0001,
 'layer_nodes': [512, 256, 128, 128, 64, 64, 32],
 'log_dir': 'logs/',
 'name': 'test_5',
 'num_freq_channels': 1024,
 'threshold': 0.00625}
Creating Network GraphINFO:tensorflow:Restoring parameters from logs/test_4/trained_model.ckpt-99
Epoch: 114 (Training, Testing) MISG: (21938.9336, 26884.6504) MSE: (0.0010, 0.0014) PWT: (17.75, 15.97)

KeyboardInterrupt: 

In [19]:
layer_nodes = [512,256,128,128,64,64,32]
test_6 = bn_FC_Network(name = 'test_6', layer_nodes = layer_nodes)
test_6.load_params('logs/test_5/params/bn_FC_Network')
test_6.initial_learning_rate = 0.00001
test_6.threshold = test_5.threshold * 2.
test_6.cost_name = 'MISG'
test_6.g_shift = 1e-5
test_6.print_params()

trainer.network = test_6
trainer.pretrained_model_path = 'logs/test_5/trained_model.ckpt-99'
trainer.fcl_keep_prob = 0.50
trainer.sample_keep_prob = 1.0
trainer.batch_size = 64
trainer.num_epochs = 10000 # quick example
trainer.model_save_interval = 25
trainer.train()

Loading Netowrk Parameters{'cost_name': 'MISG',
 'g_shift': 1e-05,
 'initial_learning_rate': 1e-05,
 'layer_nodes': [512, 256, 128, 128, 64, 64, 32],
 'log_dir': 'logs/',
 'name': 'test_6',
 'num_freq_channels': 1024,
 'threshold': 0.0125}
Creating Network GraphINFO:tensorflow:Restoring parameters from logs/test_5/trained_model.ckpt-99
Epoch: 885 (Training, Testing) MISG: (1447.9904, 2518.4861) MSE: (0.0008, 0.0010) PWT: (43.62, 43.63)

KeyboardInterrupt: 

In [22]:
layer_nodes = [512,256,128,128,64,64,32]
test_7 = bn_FC_Network(name = 'test_7', layer_nodes = layer_nodes)
test_7.load_params('logs/test_6/params/bn_FC_Network')
test_7.initial_learning_rate = 0.00005
test_7.threshold = 0.00625
test_7.cost_name = 'MISG'
test_7.g_shift = 1e-5
test_7.print_params()

trainer.network = test_7
trainer.pretrained_model_path = 'logs/test_6/trained_model.ckpt-849'
trainer.fcl_keep_prob = 0.50
trainer.sample_keep_prob = 1.0
trainer.batch_size = 256
trainer.num_epochs = 10000 # quick example
trainer.model_save_interval = 25
trainer.train()

Loading Netowrk Parameters{'cost_name': 'MISG',
 'g_shift': 1e-05,
 'initial_learning_rate': 5e-05,
 'layer_nodes': [512, 256, 128, 128, 64, 64, 32],
 'log_dir': 'logs/',
 'name': 'test_7',
 'num_freq_channels': 1024,
 'threshold': 0.00625}
Creating Network GraphINFO:tensorflow:Restoring parameters from logs/test_6/trained_model.ckpt-849
Epoch: 1410 (Training, Testing) MISG: (6305.9678, 9657.1143) MSE: (0.0006, 0.0009) PWT: (33.87, 24.92)

KeyboardInterrupt: 

In [6]:
layer_nodes = [512,256,128,128,64,64,32]
test_8 = bn_FC_Network(name = 'test_8', layer_nodes = layer_nodes)
test_8.load_params('logs/test_7/params/bn_FC_Network')
test_8.initial_learning_rate = 0.00001
test_8.threshold = 0.00625
test_8.cost_name = 'MSE'
test_8.g_shift = 1e-5
test_8.print_params()

trainer.network = test_8
trainer.pretrained_model_path = 'logs/test_7/trained_model.ckpt-1399'
trainer.fcl_keep_prob = 0.50
trainer.sample_keep_prob = 1.0
trainer.batch_size = 256
trainer.num_epochs = 10000 # quick example
trainer.model_save_interval = 25
trainer.train()

Loading Netowrk Parameters{'cost_name': 'MSE',
 'g_shift': 1e-05,
 'initial_learning_rate': 1e-05,
 'layer_nodes': [512, 256, 128, 128, 64, 64, 32],
 'log_dir': 'logs/',
 'name': 'test_8',
 'num_freq_channels': 1024,
 'threshold': 0.00625}
Creating Network GraphINFO:tensorflow:Restoring parameters from logs/test_7/trained_model.ckpt-1399
Epoch: 3848 (Training, Testing) MISG: (10992.4238, 10964.0605) MSE: (0.0007, 0.0008) PWT: (30.00, 31.40)

InvalidArgumentError: Nan in summary histogram for: logging/predictions
	 [[Node: logging/predictions = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](logging/predictions/tag, prediction/fully_connected/Relu)]]

Caused by op u'logging/predictions', defined at:
  File "/Applications/anaconda3/envs/py27/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Applications/anaconda3/envs/py27/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Applications/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Applications/anaconda3/envs/py27/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Applications/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/

In [27]:
1

1